Criação do blockchain

In [28]:
import hashlib #importa a biblioteca que faz calculo da hash
import json #importa o modulo que manipula arquivos json
from time import time #importa a função tim para obter timestamp (data e hora)

class Bloco:
  def __init__ (self, index, dados, hash_anterior = ''):
    # inicializar o bloco com os parametros fornecidos e atributos necessarios
    self.index = index #indice do bloco da cadeia
    self.timestamp = time() # registrar o momento da criação do bloco
    self.dados = dados # informações, payload (ex: transações)
    self.hash_anterior = hash_anterior # hash do bloco anterior da cadeia
    self.nonce = 0 # valor usado na prova de trabalho para alterar o hash
    self.hash_atual =self.gerar_hash() #gerar o has do bloco

  def gerar_hash (self):
    #Função para gerar o hash SHA-256 do bloco
    conteudo_bloco = json.dumps({
        'index': self.index,
        'timestamp':self.timestamp,
        'dados': self.dados,
        'hash_anterior' : self.hash_anterior,
        'nonce':self.nonce
    },sort_keys = True).encode() # converte os dados em uma string JSON ordenada
    return hashlib.sha256(conteudo_bloco).hexdigest()# calcula o hash SHA256 do conteudo do bloco

  def prova_de_trabalho(self, dificuldade):
    #realiza a prova de trabalho ajustando o nonce ate que o has hcomece com 'dificuldade' zero
    while self.hash_atual[:dificuldade] != '0'* dificuldade:
      self.nonce += 1 #incrementa o nonce para alterar o hash
      self.hash_atual = self.gerar_hash() #recalcula o hash com novo nonce

class contrato_inteligente:
  def validar_glicemia(self, dados):
        # Regra: glicemia deve estar entre 0 e 600
        if not isinstance(dados.get('glicose'), (int, float)) or not (0 <= dados['glicose'] <= 126):
            raise ValueError("Dados de glicemia inválidos! Devem ser entre 0 e 126, diabéticos não aceitos!")
        return True

class Blockchain:
  def __init__(self,contrato_inteligente):
    #inicializa a blockchain com um bloco genesis e definir a fidiculdade da prova de trabalho
    self.cadeia = [self.criar_bloco_genesis()] #lista que armazena os blocos na cadeia
    self.dificuldade = 2 # defini o numero de zeros necessarios no hash da prova de tragalho
    self.contrato = contrato_inteligente # Armazena o contrato inteligente

  def validacao(self, data):
     # Validação de glicemia (exemplo: valores aceitáveis entre 0 e 600)
     return isinstance(data, (int, float)) and 0 <= data <= 600

  def criar_bloco_genesis(self):
    return Bloco(0, "Bloco Genesis", "0") #cria o bloco genesis

  def obter_ultimo_bloco(self):
    #retura o ultimo bloco da cadeia
    return self.cadeia[-1]

  def adicionar_bloco(self, novo_bloco):
    # Valida os dados do bloco usando o contrato inteligente
    self.contrato.validar_glicemia(novo_bloco.dados)
    #adiciona novo bloco à cadeia
    novo_bloco.hash_anterior = self.obter_ultimo_bloco().hash_atual #define o hash do bloco anterior
    novo_bloco.prova_de_trabalho(self.dificuldade) # realiza a prova de trabaho para alterar o hash
    self.cadeia.append(novo_bloco) # adiciona o novo bloco à cadeia

  def validar_cadeia(self):
    #valida a integridade da cadeia, validando hashes e encademamento
    for i in range(1, len(self.cadeia)):
      bloco_atual = self.cadeia[i]
      bloco_anterior = self.cadeia[i-1]

      #verifica se o hash armazenado corresponde ao hash calculado
      if bloco_atual.hash_atual!=  bloco_atual.gerar_hash():
        return False # retorna false se houver discrepancia

      #verificar se o hash do bloco anterior corresponde ao hash armazenado no bloco anterior
      if bloco_atual.hash_anterior != bloco_anterior.gerar_hash():
        return False # retorna false se houver discrepancia
    return True # retorna true se a cadeia for validada

  def salvar_em_json(self, nome_arquivo = 'blockchain.json'):
    #salvar a cadeia em um arquivo JSON
    with open(nome_arquivo,'w') as arquivo:
      json.dump([bloco.__dict__ for bloco in self.cadeia], arquivo, indent = 4)

def coletar_dados_exame():
  glicose = float(input("Digite o valor da glicose: "))
  return{
      'glicose': glicose
  } #retorna um dicionario com os dados coletados

Instanciar Blokchain

In [29]:
# Criar o contrato inteligente
contrato = contrato_inteligente()

# Criar a blockchain com o contrato inteligente
blockchain = Blockchain(contrato)

Adicionar um bloco

In [30]:
# Loop para adicionar blocos
while True:
    try:
        dados = coletar_dados_exame()  # Coletar os dados
        novo_bloco = Bloco(len(blockchain.cadeia), dados)  # Criar o bloco
        blockchain.adicionar_bloco(novo_bloco)  # Adicionar à blockchain
        print("Bloco adicionado com sucesso!")
    except ValueError as e:
        print(e)

    continuar = input("Deseja adicionar outro bloco? (s/n): ")
    if continuar.lower() != 's':
        break

# Exibir a blockchain
for bloco in blockchain.cadeia:
    print(vars(bloco))

# Salvar a blockchain em um arquivo JSON
blockchain.salvar_em_json()
print("Blockchain salva em 'blockchain.json'.")

Digite o valor da glicose: 300
Dados de glicemia inválidos! Devem ser entre 0 e 126, diabéticos não aceitos!
Deseja adicionar outro bloco? (s/n): s
Digite o valor da glicose: 100
Bloco adicionado com sucesso!
Deseja adicionar outro bloco? (s/n): n
{'index': 0, 'timestamp': 1733792843.5815911, 'dados': 'Bloco Genesis', 'hash_anterior': '0', 'nonce': 0, 'hash_atual': '2a1d1d64e5866c12e19656acfb922dbfb8fa1bb3a131eefa0451a56220eedde9'}
{'index': 1, 'timestamp': 1733792863.4120193, 'dados': {'glicose': 100.0}, 'hash_anterior': '2a1d1d64e5866c12e19656acfb922dbfb8fa1bb3a131eefa0451a56220eedde9', 'nonce': 287, 'hash_atual': '000e6ed8f2bf376531739f978f9fce99349b62b5f36ffe2c28f7984aad60f1df'}
Blockchain salva em 'blockchain.json'.


Exibir o conteudo do arquivo Json

In [31]:
def print_json_content(filename):
  #tenta abrir e processar o arquivo json
  try:
    with open(filename,'r') as file:
      data = json.load(file)
      print(json.dumps(data, indent =4))
  except FileNotFoundError:
    print(f"Erro: Arquivo {filename} não encontrado")
  except json.JSONDecodeError:
    print(f"Erro: Arquivo {filename} não é um json válido")

print_json_content('blockchain.json')


[
    {
        "index": 0,
        "timestamp": 1733792843.5815911,
        "dados": "Bloco Genesis",
        "hash_anterior": "0",
        "nonce": 0,
        "hash_atual": "2a1d1d64e5866c12e19656acfb922dbfb8fa1bb3a131eefa0451a56220eedde9"
    },
    {
        "index": 1,
        "timestamp": 1733792863.4120193,
        "dados": {
            "glicose": 100.0
        },
        "hash_anterior": "2a1d1d64e5866c12e19656acfb922dbfb8fa1bb3a131eefa0451a56220eedde9",
        "nonce": 287,
        "hash_atual": "000e6ed8f2bf376531739f978f9fce99349b62b5f36ffe2c28f7984aad60f1df"
    }
]
